In [1]:
import glob
import pandas as pd
from preprocess import preprocess_profiles

### Preprocessing parameters

In [2]:
MAD_EPSILON = 0.0
INC_IMAGE_FEATURES = True
FEAT_SELECT_OPS = ["variance_threshold", "correlation_threshold", "drop_na_columns", "blocklist"]

### List plates with overlapping ORFs

See https://github.com/jump-cellpainting/jump-cellpainting/issues/78#issuecomment-805942281

In [3]:
JCP_OVERLAP_PLATES = [
    "OKA05.06.07.08.A",
    "OAB84.85.86.87.A",
    "OAA97.98.99.XX.A",
    "OAB25.26.27.28.A",
    "OAB41.OAC17.OAB78.79.A",
    "OAA49.59.79.80.A",
    "OAA58.60.61.62.A",
    "OAA85.86.87.88.A",
    "OAB13.14.15.16.A",
    "OAB33.34.35.36.A",
    "OAB37.38.39.40.A",
]

### Read platemaps

First need to pull them from https://github.com/jump-cellpainting/jump-orf-data/tree/master/metadata/platemaps

In [4]:
platemaps = glob.glob("../../jump-orf-data/metadata/platemaps/*/*.csv")
platemaps = pd.concat((pd.read_csv(f) for f in platemaps), ignore_index=True)
assert ~platemaps.duplicated().any()

platemaps.columns = ["Metadata_Plate", "Metadata_Plate_Map_Name"]
platemaps

,Metadata_Plate,Metadata_Plate_Map_Name
0,BR00126544,control
1,BR00121426,JUMP-Target-2_compound_platemap
2,BR00126542,OAB37.38.39.40.A
3,BR00126541,OAB37.38.39.40.A
4,BR00126540,OAB37.38.39.40.A
...,...,...
272,BR00123627,OAA41.42.43.OAB45.A
273,BR00123628,OAA41.42.43.OAB45.A
274,BR00123629,OAA41.42.43.OAB45.A
275,BR00123621,OAA41.42.43.OAB45.A


See distribution of number of replicates per plate map

In [5]:
platemaps.groupby("Metadata_Plate_Map_Name").count().value_counts()

Metadata_Plate
5                 41
6                  2
10                 2
4                  1
14                 1
22                 1
dtype: int64

### Select first 2 plates for each overlapping platemap

In [6]:
# select only JCP overlap plates
overlap_plates = platemaps[platemaps["Metadata_Plate_Map_Name"].isin(JCP_OVERLAP_PLATES)]

# select first 2 plates from each plate map
overlap_plates = overlap_plates.groupby("Metadata_Plate_Map_Name").head(2)

overlap_plates

,Metadata_Plate,Metadata_Plate_Map_Name
2,BR00126542,OAB37.38.39.40.A
3,BR00126541,OAB37.38.39.40.A
41,BR00125164,OAA58.60.61.62.A
42,BR00125163,OAA58.60.61.62.A
63,BR00121556,OAA97.98.99.XX.A
64,BR00121555,OAA97.98.99.XX.A
103,BR00123787,OAB41.OAC17.OAB78.79.A
104,BR00123786,OAB41.OAC17.OAB78.79.A
142,BR00117041,OAB84.85.86.87.A
143,BR00117040,OAB84.85.86.87.A


### Read metadata

In [7]:
wells = pd.read_csv("../../datasets/metadata/well.csv.gz")
orf = pd.read_csv("../../datasets/metadata/orf.csv.gz")

### Read profiles

In [8]:
profiles = pd.read_parquet("input/raw_profiles.parquet")
profiles.head()

,Metadata_Source,Metadata_Plate,Metadata_Well,Cells_AreaShape_Area,Cells_AreaShape_BoundingBoxArea,Cells_AreaShape_BoundingBoxMaximum_X,Cells_AreaShape_BoundingBoxMaximum_Y,Cells_AreaShape_BoundingBoxMinimum_X,Cells_AreaShape_BoundingBoxMinimum_Y,Cells_AreaShape_Center_X,...,Nuclei_Texture_Variance_RNA_5_00_256,Nuclei_Texture_Variance_RNA_5_01_256,Nuclei_Texture_Variance_RNA_5_02_256,Nuclei_Texture_Variance_RNA_5_03_256,dataset,source,workspace,profiles,batch,plate
0,source_4,BR00117035,A01,5834.000000,12214.0,569.679993,552.330017,460.510010,444.850006,515.229980,...,34.516998,35.355000,34.396000,35.499001,2021_04_26_Batch1,BR00117035,None,None,None,None
1,source_4,BR00117035,A02,5463.799805,11298.0,573.630005,521.140015,469.579987,416.619995,521.219971,...,41.456001,42.639999,41.368000,42.675999,2021_04_26_Batch1,BR00117035,None,None,None,None
2,source_4,BR00117035,A03,5416.000000,10838.0,602.969971,526.530029,501.630005,423.839996,551.570007,...,43.127998,44.056000,43.116001,44.318001,2021_04_26_Batch1,BR00117035,None,None,None,None
3,source_4,BR00117035,A04,5949.799805,12099.0,606.369995,542.229980,498.839996,433.579987,552.299988,...,48.040001,49.353001,47.851002,49.601002,2021_04_26_Batch1,BR00117035,None,None,None,None
4,source_4,BR00117035,A05,5820.700195,11727.0,574.479980,538.989990,469.359985,430.269989,521.330017,...,37.455002,38.362000,37.421001,38.554001,2021_04_26_Batch1,BR00117035,None,None,None,None


### Remove extra schema columns

See https://github.com/broadinstitute/position-effect-correction/pull/4#discussion_r1158626931

In [9]:
profiles.drop(columns=['dataset', 'source', 'workspace', 'profiles', 'batch', 'plate'], inplace=True)
profiles.columns

Index(['Metadata_Source', 'Metadata_Plate', 'Metadata_Well',
       'Cells_AreaShape_Area', 'Cells_AreaShape_BoundingBoxArea',
       'Cells_AreaShape_BoundingBoxMaximum_X',
       'Cells_AreaShape_BoundingBoxMaximum_Y',
       'Cells_AreaShape_BoundingBoxMinimum_X',
       'Cells_AreaShape_BoundingBoxMinimum_Y', 'Cells_AreaShape_Center_X',
       ...
       'Nuclei_Texture_Variance_RNA_10_02_256',
       'Nuclei_Texture_Variance_RNA_10_03_256',
       'Nuclei_Texture_Variance_RNA_3_00_256',
       'Nuclei_Texture_Variance_RNA_3_01_256',
       'Nuclei_Texture_Variance_RNA_3_02_256',
       'Nuclei_Texture_Variance_RNA_3_03_256',
       'Nuclei_Texture_Variance_RNA_5_00_256',
       'Nuclei_Texture_Variance_RNA_5_01_256',
       'Nuclei_Texture_Variance_RNA_5_02_256',
       'Nuclei_Texture_Variance_RNA_5_03_256'],
      dtype='object', length=4765)

### Join features with metadata (keep only selected plates)

In [10]:
metadata = orf.merge(wells, on="Metadata_JCP2022")
ann_dframe = metadata.merge(
    profiles, on=["Metadata_Source", "Metadata_Plate", "Metadata_Well"]
)
ann_dframe = ann_dframe.merge(overlap_plates, on="Metadata_Plate")
print(f"{ann_dframe.shape=}")
ann_dframe.groupby("Metadata_Plate")["Metadata_Well"].count()

ann_dframe.shape=(7904, 4778)


Metadata_Plate
BR00117040    368
BR00117041    368
BR00121555    276
BR00121556    276
BR00123616    368
BR00123617    368
BR00123786    368
BR00123787    368
BR00123947    368
BR00123948    368
BR00124766    364
BR00124767    364
BR00125163    368
BR00125164    368
BR00125620    368
BR00125621    368
BR00126057    368
BR00126060    368
BR00126398    368
BR00126399    368
BR00126541    368
BR00126542    368
Name: Metadata_Well, dtype: int64

### Remove positive controls and "bad constructs" (not used in the analysis)

Also, removing rows where `Metadata_Symbol` is NaN, see https://github.com/jump-cellpainting/morphmap/issues/42#issue-1673921734

In [11]:
ann_dframe = ann_dframe[~ann_dframe.Metadata_Symbol.isnull() & (ann_dframe.Metadata_broad_sample != "BAD CONSTRUCT") & (ann_dframe.Metadata_pert_type != "poscon")]
ann_dframe

,Metadata_JCP2022,Metadata_broad_sample,Metadata_Name,Metadata_Vector,Metadata_Transcript,Metadata_Symbol,Metadata_NCBI_Gene_ID,Metadata_Taxon_ID,Metadata_Gene_Description,Metadata_Prot_Match,...,Nuclei_Texture_Variance_RNA_10_03_256,Nuclei_Texture_Variance_RNA_3_00_256,Nuclei_Texture_Variance_RNA_3_01_256,Nuclei_Texture_Variance_RNA_3_02_256,Nuclei_Texture_Variance_RNA_3_03_256,Nuclei_Texture_Variance_RNA_5_00_256,Nuclei_Texture_Variance_RNA_5_01_256,Nuclei_Texture_Variance_RNA_5_02_256,Nuclei_Texture_Variance_RNA_5_03_256,Metadata_Plate_Map_Name
0,JCP2022_900006,ccsbBroad304_00008,ORF000425.1_TRC304.1,pLX_304,NM_001095.4,ASIC1,41,9606,acid sensing ion channel subunit 1,100.0,...,38.214001,35.651001,35.783001,35.743000,35.814999,35.964001,36.897999,36.033001,36.932999,OAB37.38.39.40.A
1,JCP2022_900176,ccsbBroad304_00189,ORF011559.1_TRC304.1,pLX_304,NM_000587.4,C7,730,9606,complement C7,100.0,...,75.275002,70.819000,71.277000,70.698997,71.414001,71.893997,74.083000,71.866997,74.101997,OAB37.38.39.40.A
2,JCP2022_900198,ccsbBroad304_00212,ORF000189.1_TRC304.1,pLX_304,NM_001745.4,CAMLG,819,9606,calcium modulating ligand,100.0,...,48.276001,45.180000,45.368000,45.063000,45.514000,45.768002,47.139999,45.698002,46.990002,OAB37.38.39.40.A
3,JCP2022_900294,ccsbBroad304_00321,ORF012872.1_TRC304.1,pLX_304,NM_001830.4,CLCN4,1183,9606,chloride voltage-gated channel 4,100.0,...,55.854000,52.627998,52.931000,52.535999,53.047001,53.473999,55.016998,53.360001,55.056999,OAB37.38.39.40.A
4,JCP2022_900295,ccsbBroad304_00322,ORF012041.1_TRC304.1,pLX_304,NM_000084.5,CLCN5,1184,9606,chloride voltage-gated channel 5,100.0,...,55.257999,51.611000,51.976002,51.645000,52.028000,52.368999,53.838001,52.412998,53.999001,OAB37.38.39.40.A
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7891,JCP2022_915130,ccsbBroad304_99991,ORFC00003.1_TRC304.1,pLX_304,promegaLuc.1,LUCIFERASE,LUCIFERASE,CONTROL,promegaLuc,100.0,...,54.084999,50.097000,50.417000,50.146999,50.351002,50.662998,52.004002,50.617001,52.028999,OAB41.OAC17.OAB78.79.A
7892,JCP2022_915131,ccsbBroad304_99994,ORFC00004.1_TRC304.1,pLX_304,LacZ.1,LacZ,LacZ,CONTROL,Hahn Lab LacZ,100.0,...,41.611000,38.584000,38.973999,38.659000,38.869999,39.041000,40.172001,39.105000,40.132999,OAB41.OAC17.OAB78.79.A
7893,JCP2022_915131,ccsbBroad304_99994,ORFC00004.1_TRC304.1,pLX_304,LacZ.1,LacZ,LacZ,CONTROL,Hahn Lab LacZ,100.0,...,62.062000,58.222000,58.604000,58.471001,58.828999,58.973000,60.722000,59.373001,60.835999,OAB41.OAC17.OAB78.79.A
7894,JCP2022_915131,ccsbBroad304_99994,ORFC00004.1_TRC304.1,pLX_304,LacZ.1,LacZ,LacZ,CONTROL,Hahn Lab LacZ,100.0,...,50.324001,46.210999,46.687000,46.363998,46.500000,46.895000,48.398998,47.015999,48.014999,OAB41.OAC17.OAB78.79.A


In [12]:
ann_dframe.filter(regex="^Metadata_").to_csv("output/subsampled_metadata.csv", index=False)

### Preprocess subset features

In [13]:
ann_dframe = preprocess_profiles(ann_dframe)
ann_dframe.shape

(7657, 643)

In [14]:
### Save preprocessed subset to parquet
ann_dframe.to_parquet("output/subset_profiles.parquet", index=False)